# Setup

Python version: 3.12.6

Primero se deben instalar todos los componentes para que el código funcione.

In [ ]:
%pip install groq
%pip install tenacity

# API Handling

Se configuran los parametros de uso de la API de groq

In [1]:
# API KEY para Bustio
GROQ_API_KEY = "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"

# API KEY para Vitali
GROQ_API_KEY = "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"

# Import libraries

In [2]:
import os

import groq
import tenacity

# Create the Groq client

In [3]:
# Create a client
client = groq.Client(
    #api_key=os.environ.get("GROQ_API_KEY"), # Si se quiere usar la API Key exportada al SO.
    api_key =GROQ_API_KEY,
)

print(client)

# Using the Client created

In [4]:
# Define a function to handle chat completion
@tenacity.retry(wait=tenacity.wait_exponential(min=1, max=10), stop=tenacity.stop_after_attempt(3))
def chat_completion(prompt, model):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {
                    'role': 'user',
                    'content': prompt,
                }
            ],
            max_tokens=4096
        )
        return response.choices[0].message.content
    except groq.RateLimitError:
        print("Rate limit exceeded. Please try again later.")
    except groq.AuthenticationError:
        print("Authentication failed. Please check your API key.")
    except groq.GroqError as e:
        print(f"Error: {e}")

# Using the Client created

In [5]:
#model = "llama3-8b-8192" # Erroneous model for rise exceptions
model = "llama3-8b-8192"
prompt = "Hello, what's the weather like today?"

chat_completion(prompt, model)

"I'm just a large language model, I don't have have access to real-time weather information. I can give you a general idea of the climate and weather patterns for different regions, but I don't have the ability to provide information about the current weather conditions.\n\nHowever, I can suggest some ways for you to check the weather if you're interested. You can check the weather forecast online or through a weather app on your phone or computer. You can also check the weather forecast on TV or radio broadcasts."

# Some examples

In [ ]:
completion = client.chat.completions.create(
    #model="llama3-8b-8192",
    model="llama-3.1-70b-versatile",
    messages=[
        {
            "role": "system",
            "content": "From the user text I need you to extract the relevant words or phrases. Explain why you select each phrase as relevant."
        },
        {
            "role": "user",
            "content": "Hillary Clinton agrees with John McCain by voting to give George Bush the benefit of the doubt on Iran."
        }
    ],
    temperature=1,
    max_tokens=1024,
    top_p=1,
    stream=True,
    stop=None,
)

for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")